In [ ]:
import os
from datetime import datetime, timedelta
import ipywidgets as widgets
import plotly.graph_objs as go
import yfinance as yf
import pandas as pd
import numpy as np
from IPython.display import display
from gamestonk_terminal import api as gst


In [ ]:
def view_chart(data, ticker):
    if ticker:
        df = gst.stocks.options.models.yfinance.get_iv_surface(ticker)
        grouped = df.groupby('dte')
        print(grouped)

        if not df.empty:
            z = []
            df.sort_values(by=['dte'])
            for name, group in grouped:
                temp = []
                for index, row in group.iterrows():
                    temp.append(row["impliedVolatility"])
                z.append(temp)
            print(df)
            
            # I do not believe this data is accurate, but it shows how we can look to plot
            fig = go.Figure(data=[go.Surface(z=z, x=df.dte.drop_duplicates(), y=df.strike.drop_duplicates())])
            
            fig.update_layout(title=f'{data.upper()} Surface for {ticker.upper()}', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))

            if os.environ.get("SERVER_SOFTWARE", "jupyter").startswith("voila"):
                fig.show(config={"showTips": False}, renderer="notebook")
            else:
                fig.show(config={"showTips": False})


w_auto = widgets.Layout(width="auto")
opts = ["IV", "OI", "LP"]
data_widget = widgets.Dropdown(
    options=opts, value="IV", description="Data"
)

tickers_widget = widgets.Text(description="Ticker", value="TSLA")

controls = widgets.VBox([tickers_widget, data_widget])

stocks_view = widgets.interactive_output(
    view_chart,
    {
        "data": data_widget,
        "ticker": tickers_widget,
    },
)

title_html = "<h1>Option Surface Dashboard</h1>"
app_contents = [
    widgets.HTML(title_html),
    controls,
    stocks_view,
]
app = widgets.VBox(app_contents)
display(app)